In [2]:
# Installing Necessary Packages
#Pandas, Azure-storage-blob, dotenv
!python -m pip install pandas

In [3]:
!python -m pip install azure-storage-blob

  Using cached azure_storage_blob-12.20.0-py3-none-any.whl.metadata (26 kB)
   ---------------------------------------- 0.0/392.2 kB ? eta -:--:--
   ---- ---------------------------------- 41.0/392.2 kB 991.0 kB/s eta 0:00:01
   ---- ---------------------------------- 41.0/392.2 kB 991.0 kB/s eta 0:00:01
   ---- ---------------------------------- 41.0/392.2 kB 991.0 kB/s eta 0:00:01
   ---------- --------------------------- 112.6/392.2 kB 504.4 kB/s eta 0:00:01
   ---------- --------------------------- 112.6/392.2 kB 504.4 kB/s eta 0:00:01
   ---------- --------------------------- 112.6/392.2 kB 504.4 kB/s eta 0:00:01
   ---------- --------------------------- 112.6/392.2 kB 504.4 kB/s eta 0:00:01
   ---------- --------------------------- 112.6/392.2 kB 504.4 kB/s eta 0:00:01
   ------------------ ------------------- 194.6/392.2 kB 454.0 kB/s eta 0:00:01
   ------------------ ------------------- 194.6/392.2 kB 454.0 kB/s eta 0:00:01
   ------------------ ------------------- 194.6/392.2

In [5]:
!python -m pip install python-dotenv

In [7]:
# Importing Necessary Libraries
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os


In [8]:
# Data Extraction

try:
    data = pd.read_csv(r'zipco_transaction.csv')
    print('Data Extracted Successfully')
except Exception as e:
    print(f"An Error occurred: {e}")

Data Extracted Successfully


In [9]:
data.head()

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\nEast Brandist...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   